<a href="https://colab.research.google.com/github/ChaitanyaEmani/FakeNewsDetection---Classical-ML/blob/main/Fake_News_Detection_with_Classical_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset


Dataset URL: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
License(s): CC-BY-NC-SA-4.0
  0% 0.00/41.0M [00:00<?, ?B/s]
100% 41.0M/41.0M [00:00<00:00, 1.19GB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/fake-and-real-news-dataset.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [ ]:
fakeData = pd.read_csv('/content/Fake.csv')
realData = pd.read_csv('/content/True.csv')

In [ ]:
fakeData.shape

(23481, 4)

In [ ]:
realData.shape

(21417, 4)

In [ ]:
fakeData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [ ]:
fakeData.isnull().sum()

,0
title,0
text,0
subject,0
date,0


In [ ]:
realData.isnull().sum()

,0
title,0
text,0
subject,0
date,0


In [ ]:
realData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [ ]:
realData['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [ ]:
fakeData['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [ ]:
fakeData['title'][0]

' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing'

In [ ]:
realData['title'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script'

In [ ]:
fakeData['title'][99]

' Trump Laughs After Philippine Leader Calls Journalists ‘Spies’ In Country Where Journalists Are Routinely Murdered'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Add labels
realData['label'] = 1   # Real news → 1
fakeData['label'] = 0   # Fake news → 0

# Step 2: Combine datasets
df = pd.concat([realData, fakeData], axis=0).reset_index(drop=True)

# Step 3: Shuffle the data
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0
...,...,...,...,...,...
44893,Nigeria says U.S. agrees delayed $593 million ...,ABUJA (Reuters) - The United States has formal...,worldnews,"December 27, 2017",1
44894,Boiler Room #62 – Fatal Illusions,Tune in to the Alternate Current Radio Network...,Middle-east,"June 29, 2016",0
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,I m convinced the Freedom From Religion group...,Government News,"Feb 27, 2016",0
44896,Republican tax plan would deal financial hit t...,WASHINGTON (Reuters) - The Republican tax plan...,politicsNews,"November 2, 2017",1


In [ ]:
df['label'].value_counts()

,count
label,
0,23481
1,21417


Real News --> 1
Fake News --> 0

In [ ]:
df['news'] = df['title'] + " " + df['text']

df

,title,text,subject,date,label,news
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0,BREAKING: GOP Chairman Grassley Has Had Enoug...
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0,Failed GOP Candidates Remembered In Hilarious...
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0,Mike Pence’s New DC Neighbors Are HILARIOUSLY...
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1,California AG pledges to defend birth control ...
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0,AZ RANCHERS Living On US-Mexico Border Destroy...
...,...,...,...,...,...,...
44893,Nigeria says U.S. agrees delayed $593 million ...,ABUJA (Reuters) - The United States has formal...,worldnews,"December 27, 2017",1,Nigeria says U.S. agrees delayed $593 million ...
44894,Boiler Room #62 – Fatal Illusions,Tune in to the Alternate Current Radio Network...,Middle-east,"June 29, 2016",0,Boiler Room #62 – Fatal Illusions Tune in to t...
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,I m convinced the Freedom From Religion group...,Government News,"Feb 27, 2016",0,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...
44896,Republican tax plan would deal financial hit t...,WASHINGTON (Reuters) - The Republican tax plan...,politicsNews,"November 2, 2017",1,Republican tax plan would deal financial hit t...


In [ ]:
df['news'][0]

' BREAKING: GOP Chairman Grassley Has Had Enough, DEMANDS Trump Jr. Testimony Donald Trump s White House is in chaos, and they are trying to cover it up. Their Russia problems are mounting by the hour, and they refuse to acknowledge that there are problems surrounding all of this. To them, it s  fake news,  or a  hoax.  However, the facts bear things out differently, and it seems that there are now cracks in the Congressional public leadership.Chuck Grassley (R-Iowa), who heads the Senate Judiciary Committee, is fed up. He is now demanding that Donald Trump, Jr. and former 2016 Trump Campaign Manager Paul Manafort testify before his committee regarding the now infamous shady meeting between Donald Trump and the shady Russian lawyer who promised dirt on 2016 Democratic Presidential nominee Hillary Clinton. In fact, this information is due, well, NOW. This demand sends a few signals to team Trump   most notably that they should not fire Special Counsel Robert Mueller under any circumstan

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Setup
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Precompile regexes
punct_translator = str.maketrans('', '', string.punctuation)
non_alpha_re = re.compile(r'[^a-zA-Z\s]')

def fast_preprocess(text):
    # Lowercase and strip
    text = text.lower().strip()

    # Remove non-alphabetic characters
    text = non_alpha_re.sub(' ', text)

    # Remove punctuation fast
    text = text.translate(punct_translator)

    # Tokenize and clean
    words = text.split()
    cleaned = [stemmer.stem(word) for word in words if word not in stop_words]

    return ' '.join(cleaned)


In [ ]:
df['news'] = df['news'].apply(fast_preprocess)

In [ ]:
df

,title,text,subject,date,label,news
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0,break gop chairman grassley enough demand trum...
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0,fail gop candid rememb hilari mock eulog video...
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0,mike penc new dc neighbor hilari troll homopho...
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1,california ag pledg defend birth control insur...
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0,az rancher live us mexico border destroy nanci...
...,...,...,...,...,...,...
44893,Nigeria says U.S. agrees delayed $593 million ...,ABUJA (Reuters) - The United States has formal...,worldnews,"December 27, 2017",1,nigeria say u agre delay million fighter plane...
44894,Boiler Room #62 – Fatal Illusions,Tune in to the Alternate Current Radio Network...,Middle-east,"June 29, 2016",0,boiler room fatal illus tune altern current ra...
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,I m convinced the Freedom From Religion group...,Government News,"Feb 27, 2016",0,atheist sue governor texa display capitol grou...
44896,Republican tax plan would deal financial hit t...,WASHINGTON (Reuters) - The Republican tax plan...,politicsNews,"November 2, 2017",1,republican tax plan would deal financi hit u u...


In [ ]:
X = df['news'].values
y = df['label'].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2, random_state=2)


In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(35918, 81550)
(8980, 81550)
(35918,)
(8980,)


In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5526930 stored elements and shape (35918, 81550)>
  Coords	Values
  (0, 64306)	0.3115464292430602
  (0, 54707)	0.10750520519399494
  (0, 55931)	0.2029726606279363
  (0, 1961)	0.17244012573428283
  (0, 64434)	0.38511771720488364
  (0, 54628)	0.28409121841518964
  (0, 57549)	0.3388385817987393
  (0, 66759)	0.23723421756832577
  (0, 11200)	0.2396709578530956
  (0, 78179)	0.25140298979483
  (0, 3910)	0.14875622547920167
  (0, 24359)	0.0860402475785708
  (0, 48828)	0.10279808162891468
  (0, 11894)	0.14247027015095323
  (0, 62083)	0.12115820543602127
  (0, 74319)	0.15000190323849794
  (0, 32229)	0.12089590526780984
  (0, 23348)	0.18497919754904912
  (0, 45675)	0.17696795947779906
  (0, 51119)	0.11147454959356433
  (0, 52557)	0.07500272513474661
  (0, 67916)	0.11749298495274366
  (0, 55472)	0.07832510628335343
  (0, 58002)	0.1706546450627028
  (0, 76332)	0.07849772525558942
  :	:
  (35917, 64294)	0.02509374912899344
  (35917, 75488

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
X_train = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train, Y_train)
print(training_data_accuracy)

0.9912857063310875


In [ ]:
X_test_pred = model.predict(X_test)
training_data_accuracy = accuracy_score(Y_test, X_test_pred)
training_data_accuracy

0.9875278396436525



*   Training Data Performance = 99%
*   Testing Data Performance = 98%

Both are above 70% and they are almost similar. So it is good fit.



In [ ]:
def predict_multiple(news_list):
    # Step 1: Preprocess each article
    cleaned = [fast_preprocess(news) for news in news_list]

    # Step 2: Vectorize the batch
    vectors = vectorizer.transform(cleaned)

    # Step 3: Predict
    preds = model.predict(vectors)

    # Step 4: Return results as readable labels
    results = ["Real News 🟢" if p == 1 else "Fake News 🔴" for p in preds]
    return results


In [ ]:
sample_news = df['news'][99:110]

res = predict_multiple(sample_news)
print(res)

['Real News 🟢', 'Fake News 🔴', 'Fake News 🔴', 'Fake News 🔴', 'Fake News 🔴', 'Real News 🟢', 'Fake News 🔴', 'Real News 🟢', 'Fake News 🔴', 'Real News 🟢', 'Real News 🟢']
